# Setup

TO DO:
- Different boat validation set
- Bring in listed external datasets
- Move to inception net
- Normalise the data
- Look at misclassifications for further ideas


In [1]:
%matplotlib inline
%load_ext autoreload

In [2]:
import keras
import vgg16
import os
import numpy as np

from vgg16 import Vgg16
from inceptionV3 import IncepV3
from glob import glob
from keras.layers.core import Dense

Using TensorFlow backend.


In [3]:
#model = Vgg16()
model = IncepV3()

In [4]:
#path = 'data/'
path = 'data/sample_fish/'

# Create Validation Set

In [ ]:
for sub_path in os.listdir(path+'/train/'):
    g = glob(path+'train/'+sub_path+'/*.jpg')
    shuf = np.random.permutation(g)
    for i in range(int(len(g)*0.2)):
        os.rename(shuf[i], path+'valid/' + sub_path + '/' + shuf[i].split('/')[-1])

# Fit a Model

In [5]:
bs=64
noe=6

In [6]:
batches = model.get_batches(path+'train', batch_size=bs)
valid_batches = model.get_batches(path+'valid', batch_size=bs*2)

Found 11 images belonging to 8 classes.
Found 11 images belonging to 8 classes.


In [8]:
%autoreload
model.create(batches.nb_class)
model.finetune(batches, 0.001)

/Users/simonworgan/anaconda/lib/python3.5/site-packages/Keras-1.2.0-py3.5.egg/keras/applications/inception_v3.py:291: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image dimension ordering convention (`image_dim_ordering="th"`). For best performance, set `image_dim_ordering="tf"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


In [9]:
model.fit(batches, valid_batches, nb_epoch=noe)

Epoch 1/6
11/11 [==============================] - 26s - loss: 2.3271 - acc: 0.0909 - val_loss: 2.4919 - val_acc: 0.0909
Epoch 2/6
11/11 [==============================] - 23s - loss: 1.0027 - acc: 0.9091 - val_loss: 2.1798 - val_acc: 0.0909
Epoch 3/6
11/11 [==============================] - 24s - loss: 0.1922 - acc: 1.0000 - val_loss: 2.0379 - val_acc: 0.1818
Epoch 4/6
11/11 [==============================] - 24s - loss: 0.1070 - acc: 1.0000 - val_loss: 1.8309 - val_acc: 0.3636
Epoch 5/6
11/11 [==============================] - 23s - loss: 0.0566 - acc: 1.0000 - val_loss: 1.7863 - val_acc: 0.3636
Epoch 6/6
11/11 [==============================] - 24s - loss: 0.0067 - acc: 1.0000 - val_loss: 1.2605 - val_acc: 0.6364


In [10]:
model.model.save_weights('incep_finetune1.h5')

Now optimise the other dense layers

In [11]:
for layer in model.model.layers[:172]:
   layer.trainable = False
for layer in model.model.layers[172:]:
   layer.trainable = True

model.compile(0.0001)

In [12]:
model.fit(batches, valid_batches, 6)

Epoch 1/6
11/11 [==============================] - 32s - loss: 0.0030 - acc: 1.0000 - val_loss: 1.6980 - val_acc: 0.4545
Epoch 2/6
11/11 [==============================] - 27s - loss: 0.0012 - acc: 1.0000 - val_loss: 1.1305 - val_acc: 0.6364
Epoch 3/6
11/11 [==============================] - 27s - loss: 0.0021 - acc: 1.0000 - val_loss: 0.8908 - val_acc: 0.7273
Epoch 4/6
11/11 [==============================] - 27s - loss: 0.0013 - acc: 1.0000 - val_loss: 1.2335 - val_acc: 0.5455
Epoch 5/6
11/11 [==============================] - 26s - loss: 2.8353e-04 - acc: 1.0000 - val_loss: 1.4275 - val_acc: 0.5455
Epoch 6/6
11/11 [==============================] - 24s - loss: 4.3111e-04 - acc: 1.0000 - val_loss: 0.8226 - val_acc: 0.6364


In [13]:
model.model.save_weights('finetune2.h5')

# Write out Predictions

In [14]:
batches, preds = model.test(path+'test_stg1', batch_size = bs*2)

Found 4 images belonging to 1 classes.


In [15]:
filenames = batches.filenames

with open("sample_sub.csv", "w") as f:
    f.write("image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n")
    for pred,fn in zip(preds, filenames):
        pred = ",".join(map(lambda x: str(x), pred))
        f.write(fn+","+pred+"\n")

# Analyse Results

In [ ]:
model.model.load_weights('finetune1.h5')

In [ ]:
batches.filenames